In [1]:
import requests
import json
import pprint
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import csv
import re
from datetime import datetime
import time
import numpy as np

# I. Lấy thông tin chung
- Lấy thông tin của tất cả các quán ăn thuộc 1 tỉnh/TP. Rộng hơn là thông tin tất cả quán ăn của nhiều tỉnh thành
- Với mỗi quán ăn, lấy những trường thông tin cơ bản như
    + name, addr, CityId, RestaurantId, RestaurantStatus <font color=green>// thông tin cơ bản</font>
    + AvgRating
    + TotalPictures, TotalSaves, TotalReviews, TotalFavourites, TotalViews, TotalCheckIns
    + CreatedOn <font color=green>//ngày shop dc tạo đầu tiên trên foody (trường này bị sao rồi, toàn mặc định)</font>
    + Latitude & Longitude <font color=green>//toạ độ</font>
    + ResUrlReviews <font color=green>//URL dẫn tới trang chứa bình luận của shop đó</font>


In [3]:
def get_single_page(page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text

    # handle errors
    try:
        json_data = json.loads(data)
        search_items = json_data['searchItems']
    except: # no data
        return []
    
    # get basic shop's info
    shop_attribs = []
    
    for shop in search_items: 
        shop_attribs.append({'ReviewUrl' : shop['ReviewUrl'],
                             'name': shop['Name'],
                             'Address': shop['Address'],
                             'districtid': shop['DistrictId'],
                             'RestaurantId': shop['Id'],
                             'RestaurantStatus': shop['Status'],
                             'avgrating' : shop['AvgRatingOriginal'],
                             'TotalPictures' : shop['TotalPictures'],
                             'TotalSaves' : shop['TotalSaves'],
                             'TotalReviews' : shop['TotalReview'],
                             'TotalFavourites' : shop['TotalFavourite'],
                             'TotalViews' : shop['TotalView'],
                             'TotalCheckIns' : shop['TotalCheckins'],
                             'Latitude'  : shop['Latitude'],
                             'Longitude'  : shop['Longitude']})
    
    return shop_attribs # [{},{},..]

def get_single_district(dist=1):
    '''get all shops in a single district'''
    page = 1
    all_shop = []
    empty = False

    for page in range(1, 101):
        if empty:
            break
        for st in range(1, 60):
            time.sleep(1)
            one_page = get_single_page(page, st, dist) 
            
            if not one_page: # check empty
                empty = True
                break
            all_shop.extend(one_page)
    return all_shop # [{},{},..]                       

def save_district_to_file(dist):
    shops_in_a_district = get_single_district(dist)
    df = pd.DataFrame.from_dict(shops_in_a_district) 
    df.dropna(subset='ReviewUrl').drop_duplicates().to_csv(f'q_id_{dist}.csv', index = False, header=True, encoding='utf-8')

In [4]:
all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
for dist in all_dist:
    save_district_to_file(dist)
    print(f'save district_id = {dist} successfully')

save district_id = 12 successfully
save district_id = 13 successfully
save district_id = 14 successfully


## Merge data của tất cả các quận

- Merge các file quận/huyện độc lập thành 1 dataframe duy nhất `df_all` 

In [85]:
import os

df_all = pd.DataFrame()
col_order = list(pd.read_csv('crawled_all_shops/all_districts_hcm/q1.csv'))
for file in os.listdir('crawled_all_shops/all_districts_hcm/'):
    df_dist = pd.read_csv(f"crawled_all_shops/all_districts_hcm/{file}")
    df_dist.reindex(col_order, axis=1) # df_dist's col order must be unified
    df_all = pd.concat([df_all, df_dist])

In [122]:
print(df_all.shape)
print("drop_dup(ReviewUrl) + drop_na(ReviewUrl):", df_all.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl').shape)

(58365, 15)
drop_dup(ReviewUrl) + drop_na(ReviewUrl): (58365, 15)


In [104]:
# rename columns
df_all = df_all.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl')
df_all.rename(columns={'name': 'Name', 'avgrating': 'AvgRating', 'districtid': 'District'}, inplace=True)
# đổi df['District'] từ mã encode quận sang quận bằng chữ
all_dist_encode = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')
all_dist = 'Quận 1,Quận 2,Quận 3,Quận 4,Quận 5,Quận 6,Quận 7,Quận 8,Quận 9,Quận 10,Quận 11,Quận 12,Bình Thạnh,Tân Bình,Phú Nhuận,Tân Phú,Gò Vấp,Bình Tân,Tp. Thủ Đức,Huyện Bình Chánh,Huyện Nhà Bè,Huyện Hóc Môn,Huyện Củ Chi,Huyện Cần Giờ'.split(',')
df_all['District'] = df_all['District'].apply(lambda x: all_dist[all_dist_encode.index(str(int(x)))] if x == x else "")
df_all.head()

df_all.to_csv(f'crawled_all_shops/all_dist_vers_1.csv', index = False, header=True, encoding='utf-8')
df_all.head()

- Merge chung với file `all_dist_vers_2.csv` đang chứa 69k+ dòng

In [53]:
# đọc Data_TPHCM & bsung các gtrị rỗng vào các empty_cols
data_hcm = pd.read_csv('crawled_all_shops/all_dist_vers_2.csv')[:-1] # drop 'city' col
empty_cols = list(set(list(df_all)).difference(set(list(data_hcm))))
data_hcm[empty_cols] = np.nan 
data_hcm = data_hcm.reindex(list(df_all), axis=1)

print(data_hcm.shape)
data_hcm.head()

(69186, 15)


,ReviewUrl,Name,Address,District,RestaurantId,RestaurantStatus,AvgRating,TotalPictures,TotalSaves,TotalReviews,TotalFavourites,TotalViews,TotalCheckIns,Latitude,Longitude
0,/ho-chi-minh/crystal-jade-palace-legend-lounge...,"Crystal Jade Palace - Lotte Legend Saigon Hotel""'","Legend Hotel, 2A - 4A Tôn Đức Thắng, P. Bến Nghé'",['Quận 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/ho-chi-minh/yoshino-japanese-restaurant-lotte...,Yoshino Japanese Restaurant - Lotte Hotel Saig...,"Lotte Legend Hotel, 2A - 4A Tôn Đức Thắng, P. ...",['Quận 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/ho-chi-minh/nha-hang-san-fu-lou/binh-luan,"San Fu Lou - Ẩm Thực Quảng Đông - Lê Lai""'","Tầng Trệt, AB Tower, 76A Lê Lai, P. Bến Thành'",['Quận 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/ho-chi-minh/the-log-restaurant/binh-luan,"The LOG Restaurant""'","Rooftop, GEM Center, 8 Nguyễn Bỉnh Khiêm'",['Quận 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/ho-chi-minh/nha-hang-sorae-sushi-sake-lounge/...,"Sorae Restaurant - Lounge""'","Tầng 24 - 25, AB Tower, 76A Lê Lai'",['Quận 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Preprocess dữ liệu trước khi merge

In [54]:
# preprocess district
data_hcm['District'] = data_hcm['District'].apply(lambda x: x[2:x.find("'", 2)])
data_hcm['District']

0              Quận 1
1              Quận 1
2              Quận 1
3              Quận 1
4              Quận 1
             ...     
69181    Huyện Nhà Bè
69182    Huyện Nhà Bè
69183    Huyện Nhà Bè
69184    Huyện Nhà Bè
69185    Huyện Nhà Bè
Name: District, Length: 69186, dtype: object

- Hợp nhất data từ `all_dist_vers_1` & `all_dist_vers_2`

In [55]:
# merge data
data_hcm = pd.concat([data_hcm, df_all])
data_hcm = data_hcm.drop_duplicates('ReviewUrl').dropna(subset='ReviewUrl')
data_hcm.shape

(93669, 15)

In [56]:
data_hcm.head(3)

,ReviewUrl,Name,Address,District,RestaurantId,RestaurantStatus,AvgRating,TotalPictures,TotalSaves,TotalReviews,TotalFavourites,TotalViews,TotalCheckIns,Latitude,Longitude
0,/ho-chi-minh/crystal-jade-palace-legend-lounge...,"Crystal Jade Palace - Lotte Legend Saigon Hotel""'","Legend Hotel, 2A - 4A Tôn Đức Thắng, P. Bến Nghé'",Quận 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/ho-chi-minh/yoshino-japanese-restaurant-lotte...,Yoshino Japanese Restaurant - Lotte Hotel Saig...,"Lotte Legend Hotel, 2A - 4A Tôn Đức Thắng, P. ...",Quận 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/ho-chi-minh/nha-hang-san-fu-lou/binh-luan,"San Fu Lou - Ẩm Thực Quảng Đông - Lê Lai""'","Tầng Trệt, AB Tower, 76A Lê Lai, P. Bến Thành'",Quận 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Cập nhật dữ liệu: vì `all_dist_vers_2` không chứa đủ thuộc tính như `all_dist_vers_1` nhưng có thể có trùng khoá chính `ReviewUrl`, vì vậy ta có thể cập nhật những dòng thiếu kiểu này dựa vào `all_dist_vers_1` 

In [59]:
data_hcm.set_index('ReviewUrl', inplace=True)
data_hcm.update(df_all.set_index('ReviewUrl'))
data_hcm.reset_index()

,ReviewUrl,Name,Address,District,RestaurantId,RestaurantStatus,AvgRating,TotalPictures,TotalSaves,TotalReviews,TotalFavourites,TotalViews,TotalCheckIns,Latitude,Longitude
0,/ho-chi-minh/crystal-jade-palace-legend-lounge...,Crystal Jade Palace - Lotte Legend Saigon Hotel,"Legend Hotel, 2A - 4A Tôn Đức Thắng, P. Bến Nghé",Quận 1,5.469000e+03,2.0,9.548,2575.0,0.0,601.0,286.0,62169.0,27.0,10.778231,106.706997
1,/ho-chi-minh/yoshino-japanese-restaurant-lotte...,Yoshino Japanese Restaurant - Lotte Hotel Saigon,"Lotte Legend Hotel, 2A - 4A Tôn Đức Thắng, P. ...",Quận 1,1.844000e+03,2.0,7.466,67.0,0.0,9.0,13.0,7980.0,3.0,10.778231,106.706997
2,/ho-chi-minh/nha-hang-san-fu-lou/binh-luan,San Fu Lou - Ẩm Thực Quảng Đông - Lê Lai,"Tầng Trệt, AB Tower, 76A Lê Lai, P. Bến Thành",Quận 1,9.001800e+04,2.0,7.804,4953.0,0.0,593.0,919.0,147618.0,160.0,10.770537,106.694305
3,/ho-chi-minh/the-log-restaurant/binh-luan,The LOG Restaurant,"Rooftop, GEM Center, 8 Nguyễn Bỉnh Khiêm",Quận 1,1.076090e+05,2.0,7.886,3135.0,0.0,251.0,893.0,208001.0,95.0,10.790285,106.701944
4,/ho-chi-minh/nha-hang-sorae-sushi-sake-lounge/...,Sorae Restaurant - Lounge,"Tầng 24 - 25, AB Tower, 76A Lê Lai",Quận 1,9.117900e+04,2.0,8.168,2753.0,0.0,251.0,704.0,150476.0,95.0,10.770537,106.694305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93664,/ho-chi-minh/tra-sua-agi-tra-sua-an-vat-tan-ky...,Trà Sữa Agi - Trà Sữa & Ăn Vặt - Tân Kỳ Tân Quý,"159 Tân Kỳ Tân Quý, P. Tân Sơn Nhì",Tân Phú,1.000030e+09,2.0,0.000,0.0,0.0,0.0,0.0,17.0,0.0,10.802464,106.627447
93665,/ho-chi-minh/tra-sua-laxy-thach-lam.ykdd0b/bin...,Trà Sữa Laxy - Thạch Lam,"34 Thạch Lam, P. Phú Thạnh",Tân Phú,1.000046e+09,2.0,0.000,0.0,0.0,0.0,0.0,12.0,0.0,10.776756,106.631917
93666,/ho-chi-minh/vegan-coffee-tra-sua-thai-nuoc-ra...,VEGAN COFFEE - Trà Sữa Thái & Nước Rau Má - Đư...,"15 Đường D10, P. Tây Thạnh",Tân Phú,1.000047e+09,2.0,0.000,0.0,0.0,0.0,0.0,23.0,0.0,10.815292,106.625296
93667,/ho-chi-minh/184-juice-nuoc-ep-trai-cay-sinh-t...,184 Juice - Nước Ép Trái Cây & Sinh Tố,"184 Đô Đốc Lộc, P. Tân Quý",Tân Phú,1.000047e+09,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,10.795028,106.619827


$\to$ Từ output 2 cell code trên, ta thấy những dòng trong `data_hcm` (al_dist_vers_2) có chung `ReviewUrl` với `df_all` (all_dist_vers_1) đã được cập nhật

In [60]:
# save merged data to csv
data_hcm.to_csv(f'crawled_all_shops/data_hcm.csv', index = False, header=True, encoding='utf-8')

## Khám phá dữ liệu

- Ta cần coi kỹ RestaurantId vì đây là trường dùng dê lấy thông tin menu & detail ở phần dưới

In [61]:
res_ids = data_hcm['RestaurantId'].dropna()
print('SL data thiếu RestaurantId:', data_hcm.shape[0] - res_ids.shape [0])
print('SL data trùng RestaurantId:', res_ids.duplicated().sum())

SL data thiếu RestaurantId: 35407
SL data trùng RestaurantId: 1


Tỉ lệ thiếu ở các trường trong `df_all` (tức data cào thuần json, chưa merge với file Data_TPHCM.csv)

In [65]:
df_all.dtypes

ReviewUrl            object
Name                 object
Address              object
District             object
RestaurantId        float64
RestaurantStatus    float64
AvgRating           float64
TotalPictures       float64
TotalSaves          float64
TotalReviews        float64
TotalFavourites     float64
TotalViews          float64
TotalCheckIns       float64
Latitude            float64
Longitude           float64
dtype: object

In [66]:
num_cols = df_all.select_dtypes([np.float64]).columns
def missing_ratio(s):
    return s.isna().sum() * 100 / len(s)

df_all[num_cols].agg([missing_ratio, pd.Series.min, pd.Series.max, pd.Series.nunique])

,RestaurantId,RestaurantStatus,AvgRating,TotalPictures,TotalSaves,TotalReviews,TotalFavourites,TotalViews,TotalCheckIns,Latitude,Longitude
missing_ratio,1.764756e-01,0.176476,0.176476,0.176476,0.176476,0.176476,0.176476,0.176476,0.176476,0.181616,0.179902
min,2.000000e+00,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001451,0.000000
max,1.000059e+09,2.000000,10.000000,12488.000000,338.000000,1513.000000,2982.000000,459331.000000,4308.000000,106.656876,107.099426
nunique,5.826100e+04,1.000000,2725.000000,1253.000000,83.000000,466.000000,607.000000,9486.000000,225.000000,53619.000000,53748.000000


$\to$ Dường như quán nào thiếu `RestaurantId` thì sẽ thiếu `RestaurantStatus`, `AvgRating`, `TotalPictures`, `TotalSaves`, `TotalReviews`, `TotalFavourites`, `TotalViews`, `TotalCheckIns`	vì tỉ lệ missing như nhau. Thử show 1 vài gtrị

In [67]:
df_all[df_all['RestaurantId'].isna()]

,ReviewUrl,Name,Address,District,RestaurantId,RestaurantStatus,AvgRating,TotalPictures,TotalSaves,TotalReviews,TotalFavourites,TotalViews,TotalCheckIns,Latitude,Longitude
11488,/ho-chi-minh/ha-noi-quan-bun-cha-bun-dau-bun-g...,NaN,"86 Trương Định, P. Bến Thành",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11489,/ho-chi-minh/com-tam-dem-mui-tau-214-c63-nguye...,NaN,"214/C63 Nguyễn Trãi, P. Nguyễn Cư Trinh",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11490,/ho-chi-minh/com-tam-24-xua-1990-co-bac/binh-luan,NaN,"71/7 Cô Bắc, P. Cô Giang",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11491,/ho-chi-minh/milk-story-sua-hat-nha-lam/binh-luan,NaN,"12M1 Nguyễn Thị Minh Khai, P. Đa Kao",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11492,/ho-chi-minh/beo-bo-nam-nuong-mon-ngon-ha-noi/...,NaN,"121A Hoàng Sa, P. Tân Định",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11586,/ho-chi-minh/pharaohs-bbq-egypt-restaurant-asi...,NaN,"Q15 - 16, Asiana Food Town, 4 Phạm Ngũ Lão, P....",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11587,/ho-chi-minh/hoa-dong-quan-bun-cha/binh-luan,NaN,"121 Lý Tự Trọng, P. Bến Nghé",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11588,/ho-chi-minh/mama-pho-le-thanh-ton/binh-luan,NaN,"134 Lê Thánh Tôn, P. Bến Thành",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11589,/ho-chi-minh/o-suu-2-banh-canh-bot-gao-bun-bo/...,NaN,"1 Trần Khắc Chân, P. Tân Định",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cách giải quyết: cào trực tiếp các trường này trên web bằng cách đi tới các url

# II. Lấy thông tin cụ thể từng quán ăn
Thông tin lần này chia thành 2 loại:
- Thông tin thực đơn (Get delivery dishes)
    - dish_type_name <font color=green>//Loại đồ ăn: 1 loại đồ ăn gồm nhiều món ăn, VD: bún thì có bún cá, bún chả</font>
    - total_like <font color=green>//tổng lượt like loại đồ ăn đó</font>
    - is_available <font color=green>//loại đồ ăn đó còn món ko</font>
    - dish_name <font color=green>//món ăn thuộc loại đồ ăn đó. VD: bún cá thập cẩm</font>
    - dish_price <font color=green>//giá 1 món ăn thuộc 1 loại đồ ăn </font>
    - options: <font color=green>// có option nào cho món ăn đó. VD: có option chọn thêm topping như rau, cua,... </font>
        [
        option_1: name, price;
        option_2: name, price;
        ....
        ]
    - option_min_select, option_max_select

- Thông tin chi tiết quán ăn (Get detail):
    - phone 
	- district_id
	- category <font color=green>//loại hình quán ăn?</font>
	- cuisines <font color=green>//quán ăn thuộc phân khúc món nào? món Việt, Âu,...?</font>
	- service_fee <font color=green>//fí dịch vụ</font>
	- avg_price <font color=green>// chắc là giá trung bình của quán ăn.</font>
	- min_order_value <font color=green>//fải mua tối thiểu bnhiêu tiền?</font>
	- promotion <font color=green>// trong này nhiều trường nhỏ, tự nghiên cứu thêm</font>
	- min_charge <font color=green>//fí fụ thu nhỏ nhất</font>
	- week_days, operating <font color=green>//thời gian mở cửa</font>
	- min_shipping_fee <font color=green>// có thể là giá ship thấp nhất</font>
	- is_foody_delivery <font color=green>// có thể là shop này có phải là shop bán mang về hay không</font>
	- price_range <font color=green>// khoảng giá min – max của quán.</font>

---

- Lấy **menu** của 1 quán ăn: xây dựng hàm nhận vào `id_quán_ăn` & get về thông tin menu

In [ ]:
# globals
menu_df = pd.DataFrame(columns=['dish_type_id', 'dish_type_name'])
menu_dish_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
menu_dish_option_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
menu_dish_option_item_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

def get_delivery_dish(restaurant_id):
	global menu_df, menu_dish_df, menu_dish_option_df, menu_dish_option_item_df
	delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={restaurant_id}&id_type=1", 
									 headers={'x-foody-api-version': '1',
											'x-foody-app-type': '1004',
											'x-foody-client-language': 'vi',
											'x-foody-client-type': '1',
											'x-foody-client-version': '1',
											'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
											}
									)
	data = delivery_dish_API.text
	try:
		dataJson = json.loads(data)
		menu_infos = dataJson['reply']['menu_infos']
	except:
		return

	menu_df_temp = pd.DataFrame(columns=list(menu_df))
	menu_dish_df_temp = pd.DataFrame(columns=list(menu_dish_df))
	menu_dish_option_df_temp = pd.DataFrame(columns=list(menu_dish_option_df))
	menu_dish_option_item_df_temp = pd.DataFrame(columns=list(menu_dish_option_item_df))

	for menu_info in menu_infos:
		menu_df_temp.loc[len(menu_df_temp.index)] = [restaurant_id, menu_info['dish_type_id'], menu_info['dish_type_name']]
		for dish in menu_info['dishes']:
			menu_dish_df_temp.loc[len(menu_dish_df_temp.index)] = [
					restaurant_id,
					menu_info['dish_type_id'],
					dish['id'],
					dish['name'],
					dish['description'],
					dish['price']['value'],
					dish['total_like'],
					dish['is_available']
			]
			for option in dish['options']:
				menu_dish_option_df_temp.loc[len(menu_dish_option_df_temp.index)] = [
						restaurant_id,
						menu_info['dish_type_id'],
						dish['id'],
						option['id'],
						option['option_items']['min_select'],
						option['option_items']['max_select']
				]
				for item in option['option_items']['items']:
					menu_dish_option_item_df_temp.loc[len(menu_dish_option_item_df_temp.index)] = [
							restaurant_id,
							menu_info['dish_type_id'],
							dish['id'],
							option['id'],
							item['id'],
							item['name'],
							item['price']['value']
					]
								
	menu_df = pd.concat([menu_df, menu_df_temp])
	menu_dish_df = pd.concat([menu_dish_df, menu_dish_df_temp])
	menu_dish_option_df = pd.concat([menu_dish_option_df, menu_dish_option_df_temp])
	menu_dish_option_item_df = pd.concat([menu_dish_option_item_df, menu_dish_option_item_df_temp])
# TEST
data_hcm = pd.read_csv('crawled_all_shops/data_hcm.csv')
data_hcm['RestaurantId'] = data_hcm['RestaurantId'].apply(lambda x: str(x).split('.')[0] if x == x else np.nan)
cnt = 0

for i, res_id in enumerate(data_hcm['RestaurantId']):
	if res_id == res_id: # check nan
		print(f'line {i+1}')
		get_delivery_dish(res_id)

In [3]:
# check
menu_df.shape

(81393, 3)

In [4]:
# export to csv
menu_df.to_csv('crawled_menu/menu.csv', sep=',', encoding='utf-8', index = False)
menu_dish_df.to_csv('crawled_menu/menu.dish.csv', sep=',', encoding='utf-8', index = False)
menu_dish_option_df.to_csv('crawled_menu/menu.dish.option.csv', sep=',', encoding='utf-8', index = False)
menu_dish_option_item_df.to_csv('crawled_menu/menu.dish.option.item.csv', sep=',', encoding='utf-8', index = False)

- Lấy **detail** của 1 quán ăn: xây hàm nhận vào `id_quán_ăn` và lưu thông tin chi tiết quán đó (VD: min_charge, rating_avg, min_shipping_fee,...) vào file csv

In [ ]:
detail_df = pd.DataFrame(columns=['city_id', 'district_id', 'request_id', 'categories', 'cuisines', 'total_review', 'avg', \
    'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'minimun_shiping_fee', 'is_foody_delivery', 'min_price', \
    'max_price', 'promotion'])

def get_detail(restaurant_id):
    global detail_df
    detail_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={restaurant_id}&id_type=1", 
                                headers={'x-foody-api-version': '1',
                                        'x-foody-app-type': '1004',
                                        'x-foody-client-language': 'vi',
                                        'x-foody-client-type': '1',
                                        'x-foody-client-version': '1',
                                        'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                            )
    data = detail_API.text
    try:
        dataJson = json.loads(data)
        detail_infos = dataJson['reply']['delivery_detail']
    except:
        return
    
    detail_df_temp = pd.DataFrame(columns=list(detail_df))
    detail_rating_df_temp = pd.DataFrame(columns=['request_id', 'total_review', 'avg'])
    detail_delivery_df_temp = pd.DataFrame(columns=['request_id','service_fee', 'avg_price', 'min_order_value', 'min_charge', 'minimun-shiping-fee', 'is_foody_delivery'])
    detail_price_df_temp = pd.DataFrame(columns=['request_id', 'min', 'max'])
    
    detail_df_temp.loc[len(detail_df_temp.index)] = [ \
        detail_infos['city_id'], detail_infos['district_id'], restaurant_id, detail_infos['categories'], \
        detail_infos['cuisines'], detail_infos['rating']['total_review'], detail_infos['rating']['avg'], \
        detail_infos['delivery']['service_fee']['value'],detail_infos['delivery']['avg_price']['value'], \
        detail_infos['delivery']['min_order_value']['value'], detail_infos['delivery']['min_charge'], \
        detail_infos['delivery']['shipping_fee']['minimum_fee'], detail_infos['delivery']['is_foody_delivery'], \
        detail_infos['price_range']['min_price'], detail_infos['price_range']['max_price'], detail_infos['delivery']['promotions']
    ]

    detail_df = pd.concat([detail_df, detail_df_temp])

data_hcm = pd.read_csv('crawled_all_shops/data_hcm.csv')
data_hcm['RestaurantId'] = data_hcm['RestaurantId'].apply(lambda x: str(x).split('.')[0] if x == x else np.nan)
cnt = 0

for i, res_id in enumerate(data_hcm['RestaurantId']):
    if cnt > 200:
        break 
    if res_id == res_id: # check nan
        print(f'line {i+1}')
        cnt += 1
        get_detail(res_id)

In [23]:
data_hcm['ReviewUrl'].nunique()

93669

In [22]:
detail_df['request_id'].nunique()

147

In [ ]:
detail_df.to_csv('crawled_shop_details/details.csv', sep=',', encoding='utf-8', index = False)